# Multiscale MILP with Materials and Emission

This is a continuation of 'One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming Example' [Example 2]. 

We will be adding the following considerations: 

1. material for the establishment of operations 
2. global warming potential induced as a sum of our decisions and choices 

## See Example 2

In [1]:
from energia import *

m = Model('example3')
m.q = Periods()
m.y = 4 * m.q


In [2]:
m.usd = Currency()
m.declare(Resource, ['power', 'wind', 'solar'])
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49

m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432
m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.charge.operate <= 1
m.lii.discharge.capacity <= 100
m.lii.discharge.operate <= 1

--- General Resource Balance for solar in (l, q): initializing constraint, adding consume(solar, l, q)
    Completed in 0.00015020370483398438 seconds
--- Binding consume in domain (solar, l, q)
    Completed in 7.796287536621094e-05 seconds
--- General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
    Completed in 6.389617919921875e-05 seconds
--- Binding consume in domain (wind, l, y)
    Completed in 4.291534423828125e-05 seconds
--- General Resource Balance for power in (l, q): initializing constraint, adding release(power, l, q)
    Completed in 8.416175842285156e-05 seconds
--- Binding release in domain (power, l, q)
    Completed in 0.00012159347534179688 seconds
--- Binding capacity in domain (wf, l, y)
    Completed in 9.751319885253906e-05 seconds
--- Binding capacity in domain (wf, l, y)
    Completed in 9.703636169433594e-05 seconds
--- Binding operate in domain (wf, l, q)
    Completed in 0.00014591217041015625 seconds
--- Mapping

## Indicator Stream 

Indicators scale the impact of some decision or flow and project it only a common dimension allowing the calculation of overall impact as a single metric value

In this case, we consider global warming potential (GWP).

In [3]:
m.gwp = Environ()

Now say that we want to consider the impact of setting up operations (specific to the act of construction)

These can be provided as calculations 

In [4]:
m.wf.capacity[m.gwp.emit] == 1000
m.pv.capacity[m.gwp.emit] == 2000
m.lii.capacity[m.gwp.emit] == 3000
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that impact streams are mapped across scales but not balanced, unless a negative (relatively) impact is also assessed! In the case of emission, this would be abatement.

## Material Stream

Note that all the objects found in energia.components.commodity.misc are still resources (Land, Material, etc.)

This will allow for the modeling of an expansive type of processes.

Examples:

1. land clearance to transform agricultural land into industrial, or land remediation 
2. resource intense material recycling 
3. some resource flowing being treated as emission flows 

The possibilities are vast... The advanced modeler may prefer to use Resource() for everything [see Resource Task Network (RTN) Framework]

In the following example, we consider a limit to cement consumption, as also an expense and gwp impact associated with it

In [5]:
m.cement = Material()
m.cement.consume <= 1000000
m.cement.consume[m.usd.spend] == 17
m.cement.consume[m.gwp.emit] == 0.9

--- General Resource Balance for cement in (l, y): initializing constraint, adding consume(cement, l, y)
    Completed in 0.00010085105895996094 seconds
--- Binding consume in domain (cement, l, y)
    Completed in 4.553794860839844e-05 seconds


Next, we provide details of the use of cement across all operations 

In [6]:
m.wf.capacity[m.cement.use] == 400
m.pv.capacity[m.cement.use] == 560
m.lii.capacity[m.cement.use] == 300

--- General Resource Balance for cement in (l, y): adding use(cement, l, y, capacity, wf)
    Completed in 9.059906005859375e-05 seconds
--- General Resource Balance for cement in (l, y): adding use(cement, l, y, capacity, pv)
    Completed in 6.175041198730469e-05 seconds
--- General Resource Balance for cement in (l, y): adding use(cement, l, y, invcapacity, power.lii)
    Completed in 6.222724914550781e-05 seconds


Material use is summed up across all operations, and adheres to an upper bound in terms of consumption. 

In [7]:
m.cement.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

GWP impact is now the sum of impact from material use as well as constuction 

In [8]:
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Locate Operations

In [9]:
m.network.locate(m.wf, m.pv, m.lii)

--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
    Completed in 0.00012087821960449219 seconds
--- General Resource Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
    Completed in 5.5789947509765625e-05 seconds
--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
    Completed in 8.630752563476562e-05 seconds
--- General Resource Balance for solar in (l, q): adding expend(solar, l, q, operate, pv)
    Completed in 8.225440979003906e-05 seconds
--- Assuming  power.lii inventory capacity is unbounded in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, q)
--- Mapping inventory across time from (power.lii, l, q) to (power.lii, l, y)
--- Creating map to (power.lii, l, y). Mapping inventory: from (power.lii, l, q) to (power.lii, l, y)
    Completed in 5.0067901611328125e-05 seconds
--

## Optimization 

Let us minimize GWP this time. Note that m.usd.spend.opt() can still be used!

In [10]:
m.gwp.emit.opt()

--- Creating map to (gwp, l, y). Mapping emit: from (gwp, l, y, capacity, wf) to (gwp, l, y)
    Completed in 9.226799011230469e-05 seconds
--- Mapping emit: from (gwp, l, y, capacity, pv) to (gwp, l, y)
    Completed in 5.125999450683594e-05 seconds
--- Mapping emit: from (gwp, l, y, invcapacity, power.lii) to (gwp, l, y)
    Completed in 5.269050598144531e-05 seconds
--- Mapping emit: from (gwp, l, y, consume, cement) to (gwp, l, y)
    Completed in 4.4345855712890625e-05 seconds
--- Generating example3.mps
--- Creating gurobi model for example3
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file example3.mps
Reading time = 0.00 seconds
EXAMPLE3: 95 rows, 87 columns, 217 nonzeros
--- Optimizing example3 using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 lo

In [11]:
m.power.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [12]:
m.sol(False)

# Solution for example3

<br><br>

## Objective

<IPython.core.display.Math object>

<br><br>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>